# **This notebook demonstrates how to filter medical notes from `NOTEEVENTS` by ICD-9 diagnosis code: 430**

 *ICD-9, 430, is subarachnoid hemorrhage (SAH). SAH is a life-threatening accumulation of blood between the arachnoid and pia mater in the brain.*

---

### **Steps**

1. [Pandas](https://pandas.pydata.org) is used to load `D_ICD_DIAGNOSES.csv.gz` and `NOTEEVENTS.csv.gz`.
2. Filter unique subjects with `ICD9_CODE=='430'`.
3. Additionally, filter medical notes with `CATEGORY=='Radiology'`.
4. Save DataFrame to CSV file.

---

### **References**

[Documentation for NOTEEVENTS](https://mimic.mit.edu/docs/iii/tables/noteevents/)

[Documentation for DIAGNOSES_ICD](https://mimic.mit.edu/docs/iii/tables/diagnoses_icd/)

-

P.S. This transformation can be done in a single query in SQL.

In [ ]:
import pandas as pd

In [ ]:
# RUN if files are located on your Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **Start by uploading the DIAGNOSES_ICD file and getting all patients for** `ICD9_CODE==430`

In [ ]:
# RUN if DIAGNOSES_ICD is located on your local machine
from google.colab import files
uploaded = files.upload()

In [ ]:
PATH_D_ICD = 'DIAGNOSES_ICD.csv.gz'

# PATH_D_ICD = '/content/drive/PATH_TO_DIAGNOSES_ICD.csv.gz' #edit to path on mounted drive

df_diagnoses_icd = pd.read_csv(PATH_D_ICD, compression='gzip',
                               dtype={'SEQ_NUM': 'Int64'})
df_diagnoses_icd.info()
df_diagnoses_icd.iloc[0]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 651047 entries, 0 to 651046
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   ROW_ID      651047 non-null  int64 
 1   SUBJECT_ID  651047 non-null  int64 
 2   HADM_ID     651047 non-null  int64 
 3   SEQ_NUM     651000 non-null  Int64 
 4   ICD9_CODE   651000 non-null  object
dtypes: Int64(1), int64(3), object(1)
memory usage: 25.5+ MB


,0
ROW_ID,1297
SUBJECT_ID,109
HADM_ID,172335
SEQ_NUM,1
ICD9_CODE,40301


In [ ]:
## Filter rows with ICD9 == 430
ICD_FILTER = [
    '430',
    # '4019',
    # '42731',
    # '4280',
    # '41401',
    # '5849',
]
df_filtered = df_diagnoses_icd[df_diagnoses_icd['ICD9_CODE'].isin(ICD_FILTER)]
subj_filtered = df_filtered['SUBJECT_ID'].unique()
print(f"Number of ICD-9 {ICD_FILTER} patients: {subj_filtered.size}")

Number of ICD-9 430 patients: 647


# **Now Upload NOTEEVENTS and filter rows with** `subj_filtered`

In [ ]:
# RUN if NOTEEVENTS is located on your local machine
from google.colab import files
uploaded = files.upload()

In [ ]:
PATH_TO_NOTEEVENTS = 'NOTEEVENTS.csv.gz'

# PATH_TO_NOTEEVENTS = '/content/PATH_TO_NOTEEVENTS.csv.gz' #edit to path on mounted drive

df_noteevents = pd.read_csv(PATH_TO_NOTEEVENTS, compression='gzip')
df_noteevents.info()
df_noteevents.iloc[0]

<ipython-input-7-d5cfebc53810>:6: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  df_noteevents = pd.read_csv(PATH_TO_NOTEEVENTS, compression='gzip')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2083180 entries, 0 to 2083179
Data columns (total 11 columns):
 #   Column       Dtype  
---  ------       -----  
 0   ROW_ID       int64  
 1   SUBJECT_ID   int64  
 2   HADM_ID      float64
 3   CHARTDATE    object 
 4   CHARTTIME    object 
 5   STORETIME    object 
 6   CATEGORY     object 
 7   DESCRIPTION  object 
 8   CGID         float64
 9   ISERROR      float64
 10  TEXT         object 
dtypes: float64(3), int64(2), object(6)
memory usage: 174.8+ MB


,0
ROW_ID,174
SUBJECT_ID,22532
HADM_ID,167853.0
CHARTDATE,2151-08-04
CHARTTIME,NaN
STORETIME,NaN
CATEGORY,Discharge summary
DESCRIPTION,Report
CGID,NaN
ISERROR,NaN


Sample medical note:

In [ ]:
print(df_noteevents.iloc[0]['TEXT'])

Admission Date:  [**2151-7-16**]       Discharge Date:  [**2151-8-4**]


Service:
ADDENDUM:

RADIOLOGIC STUDIES:  Radiologic studies also included a chest
CT, which confirmed cavitary lesions in the left lung apex
consistent with infectious process/tuberculosis.  This also
moderate-sized left pleural effusion.

HEAD CT:  Head CT showed no intracranial hemorrhage or mass
effect, but old infarction consistent with past medical
history.

ABDOMINAL CT:  Abdominal CT showed lesions of
T10 and sacrum most likely secondary to osteoporosis. These can
be followed by repeat imaging as an outpatient.



                            [**First Name8 (NamePattern2) **] [**First Name4 (NamePattern1) 1775**] [**Last Name (NamePattern1) **], M.D.  [**MD Number(1) 1776**]

Dictated By:[**Hospital 1807**]
MEDQUIST36

D:  [**2151-8-5**]  12:11
T:  [**2151-8-5**]  12:21
JOB#:  [**Job Number 1808**]



Optional preprocessing steps:

In [ ]:
# Fill NaN CHARTTIME with an CHARTDATE + 00:00:00 time
df_noteevents["CHARTTIME"] = df_noteevents["CHARTTIME"].fillna(df_noteevents["CHARTDATE"] + " 00:00:00")

# Additionally, ensure NaN type from STORETIME are converted to NaT (Not a Time)
df_noteevents["STORETIME"] = pd.to_datetime(df_noteevents["STORETIME"], errors="coerce")
df_noteevents['CHARTTIME']

,CHARTTIME
0,2151-08-04 00:00:00
1,2118-06-14 00:00:00
2,2119-05-25 00:00:00
3,2124-08-18 00:00:00
4,2162-03-25 00:00:00
...,...
2083175,2132-01-21 03:27:00
2083176,2132-01-21 09:50:00
2083177,2132-01-21 16:42:00
2083178,2132-01-21 18:05:00


# **Filter the data in files to create a new relevant dataframe**

In [ ]:
df_noteevents_filtered = df_noteevents[df_noteevents['SUBJECT_ID'].isin(subj_filtered)]
df_noteevents_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Index: 34258 entries, 74 to 2066473
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   ROW_ID       34258 non-null  int64         
 1   SUBJECT_ID   34258 non-null  int64         
 2   HADM_ID      31929 non-null  float64       
 3   CHARTDATE    34258 non-null  object        
 4   CHARTTIME    34258 non-null  object        
 5   STORETIME    19699 non-null  datetime64[ns]
 6   CATEGORY     34258 non-null  object        
 7   DESCRIPTION  34258 non-null  object        
 8   CGID         19699 non-null  float64       
 9   ISERROR      20 non-null     float64       
 10  TEXT         34258 non-null  object        
dtypes: datetime64[ns](1), float64(3), int64(2), object(5)
memory usage: 3.1+ MB


In [ ]:
df_noteevents_filtered['CATEGORY'].unique()

array(['Discharge summary', 'Echo', 'ECG', 'Nursing', 'Physician ',
       'General', 'Respiratory ', 'Nutrition', 'Social Work',
       'Rehab Services', 'Consult', 'Case Management ', 'Pharmacy',
       'Radiology', 'Nursing/other'], dtype=object)

### Let's say we only want the notes where `CATEGORY=='Radiology'`.

We obtain a DataFrame with 11,385 medical notes (1 MB) down from 2,083,180 (175 MB) in the original `NOTEEVENTS` table.

In [ ]:
df_430_radiology_notes = df_noteevents_filtered[df_noteevents_filtered['CATEGORY'] == 'Radiology']
df_430_radiology_notes.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11385 entries, 732908 to 1300853
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   ROW_ID       11385 non-null  int64         
 1   SUBJECT_ID   11385 non-null  int64         
 2   HADM_ID      9639 non-null   float64       
 3   CHARTDATE    11385 non-null  object        
 4   CHARTTIME    11385 non-null  object        
 5   STORETIME    0 non-null      datetime64[ns]
 6   CATEGORY     11385 non-null  object        
 7   DESCRIPTION  11385 non-null  object        
 8   CGID         0 non-null      float64       
 9   ISERROR      0 non-null      float64       
 10  TEXT         11385 non-null  object        
dtypes: datetime64[ns](1), float64(3), int64(2), object(5)
memory usage: 1.0+ MB


In [ ]:
print(df_430_radiology_notes.iloc[0]['TEXT'])

[**2138-8-20**] 9:36 PM
 CHEST (PORTABLE AP)                                             Clip # [**Clip Number (Radiology) 29153**]
 Reason: s/p aneurysm clipping. Spiked Fever of 102
 ______________________________________________________________________________
 [**Hospital 4**] MEDICAL CONDITION:
  50 year old man s/p aneurysm clipping, spiked fever of 102 F, R/O pneumonia.
 REASON FOR THIS EXAMINATION:
  s/p aneurysm clipping. Spiked Fever of 102
 ______________________________________________________________________________
                                 FINAL REPORT
 INDICATION:  Aneurysm clipping. Spike in fevers.

 CHEST, SINGLE VIEW

 Comparison to prior radiograph [**2138-8-17**].

 The heart, mediastinal and hilar contours are within normal limits. The
 patient is mildly rotated but the visualized lungs are clear with no focal
 consolidations or pleural effusions.

 IMPRESSION:

 1. No evidence of pneumonia.




Be mindful that a `DESCRIPTION=='Addendum'` indicates an addendum (additional text to be added to the previous report)—i.e., you may need to concatenate some notes together down the line.

# **Download the created DataFrame**

In [ ]:
df_430_radiology_notes.to_csv('df_430_radiology_notes.csv', index=False)